In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import xmltodict
import datetime

In [5]:
pd.options.display.max_columns = 25

## 시작날짜, 마지막 날짜 넣으면 해당 기간동안 일일 조회 기록 반환하는 함수

In [16]:
def movie_info(start_date, end_date):
    
    dt_index = pd.date_range(start = str(start_date), end = str(end_date))
    
    DataSet = []
    for date in dt_index:
        address = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.xml?'
        option = 'key=5e9e89c5d971566b3955b179d41f94bd&targetDt='
        date = str(date).split(' ')[0].replace('-', '')

        url = address + option + date

        res = requests.get(url)
        html = res.content

        xmlobj = xmltodict.parse(html)
        
        allData = xmlobj['boxOfficeResult']['dailyBoxOfficeList']['dailyBoxOffice']
        for source in allData:
            DataSet.append(source.values())
            
    return DataSet

In [4]:
movie_sep = movie_info(20200901, 20200926)
movie_aug = movie_info(20200801, 20200831)
movie_Jul = movie_info(20200701, 20200731)
movie_Jun = movie_info(20200601, 20200630)
movie_May = movie_info(20200501, 20200531)
movie_Apr = movie_info(20200401, 20200430)
movie_Mar = movie_info(20200301, 20200331)
movie_Feb = movie_info(20200201, 20200229)
movie_Jan = movie_info(20200101, 20200131)  # 국내 코로나 첫 확진자는 1월 19일

In [5]:
movie_20 = [movie_Jan, movie_Feb, movie_Mar, movie_Apr, movie_May, movie_Jun, movie_Jul, movie_aug, movie_sep]

## 공통코드 조회. 지역코드

In [6]:
address = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/code/searchCodeList.xml?'
option = 'key=5e9e89c5d971566b3955b179d41f94bd&comCode=0105000000'
url = address + option

res = requests.get(url)
html = res.content
soup = BeautifulSoup(html, 'html.parser')


In [7]:
loc_code = soup.text
loc_code

'0105001서울시0105002경기도0105003강원도0105004충청북도0105005충청남도0105006경상북도0105007경상남도0105008전라북도0105009전라남도0105010제주도0105011부산시0105012대구시0105013대전시0105014울산시0105015인천시0105016광주시0105017세종시'

In [8]:
address = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.xml?'
option = 'key=5e9e89c5d971566b3955b179d41f94bd&targetDt=20200901&itemPerPage=10&wideAreCd=0105001'

url = address + option

res = requests.get(url)
html = res.content

xmlobj = xmltodict.parse(html)

allData = xmlobj['boxOfficeResult']['dailyBoxOfficeList']['dailyBoxOffice']
allData

[OrderedDict([('rnum', '1'),
              ('rank', '1'),
              ('rankInten', '0'),
              ('rankOldAndNew', 'OLD'),
              ('movieCd', '20201122'),
              ('movieNm', '테넷'),
              ('openDt', '2020-08-26'),
              ('salesAmt', '405736600'),
              ('salesShare', '73.3'),
              ('salesInten', '-72593680'),
              ('salesChange', '-15.2'),
              ('salesAcc', '6902129920'),
              ('audiCnt', '46440'),
              ('audiInten', '-7567'),
              ('audiChange', '-14'),
              ('audiAcc', '760852'),
              ('scrnCnt', '2018'),
              ('showCnt', '6303')]),
 OrderedDict([('rnum', '2'),
              ('rank', '2'),
              ('rankInten', '0'),
              ('rankOldAndNew', 'OLD'),
              ('movieCd', '20197922'),
              ('movieNm', '다만 악에서 구하소서'),
              ('openDt', '2020-08-05'),
              ('salesAmt', '71605890'),
              ('salesShare', '12.9'),
 

## 컬럼명 뽑아놓기

In [9]:
col_name = []
for data in allData[0]:
    col_name.append(data)

In [10]:
col_name

['rnum',
 'rank',
 'rankInten',
 'rankOldAndNew',
 'movieCd',
 'movieNm',
 'openDt',
 'salesAmt',
 'salesShare',
 'salesInten',
 'salesChange',
 'salesAcc',
 'audiCnt',
 'audiInten',
 'audiChange',
 'audiAcc',
 'scrnCnt',
 'showCnt']

## 구한 값들을 합쳐서 DataFrame 형태로 만들어보자

In [11]:
len(movie_20)

9

In [12]:
df = pd.DataFrame(columns = col_name)
j = 0
for i in range(len(movie_20)):  #월 수만큼 반복
    for source in movie_20[i]:  #각 월마다 일수들을 source로
        source = list(source)   #리스트화해서 넣기
        df.loc[j] = source
        j += 1

In [13]:
df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439
2,3,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997
3,4,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614
4,5,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974
2696,7,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628
2697,8,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314
2698,9,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430


In [14]:
df.info() # openDt 말고는 값들이 잘 들어가있는 것을 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2700 entries, 0 to 2699
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rnum           2700 non-null   object
 1   rank           2700 non-null   object
 2   rankInten      2700 non-null   object
 3   rankOldAndNew  2700 non-null   object
 4   movieCd        2700 non-null   object
 5   movieNm        2700 non-null   object
 6   openDt         2687 non-null   object
 7   salesAmt       2700 non-null   object
 8   salesShare     2700 non-null   object
 9   salesInten     2700 non-null   object
 10  salesChange    2700 non-null   object
 11  salesAcc       2700 non-null   object
 12  audiCnt        2700 non-null   object
 13  audiInten      2700 non-null   object
 14  audiChange     2700 non-null   object
 15  audiAcc        2700 non-null   object
 16  scrnCnt        2700 non-null   object
 17  showCnt        2700 non-null   object
dtypes: object(18)
memory usage: 

In [15]:
del df['rnum'] # rnum은 순번이므로 삭제해줌

In [16]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430


In [17]:
df['movieNm'].unique()

array(['백두산', '천문: 하늘에 묻는다', '시동', '미드웨이', '겨울왕국 2',
       '신비아파트 극장판 하늘도깨비 대 요르문간드', '포드 V 페라리', '21 브릿지: 테러 셧다운', '나이브스 아웃',
       '눈의 여왕4', '해치지않아', '프린스 코기', '닥터 두리틀', '스타워즈: 라이즈 오브 스카이워커',
       '타발루가와 얼음공주', '스파이 지니어스', '나쁜 녀석들: 포에버', '히트맨', '타오르는 여인의 초상',
       '아내를 죽였다', '남산의 부장들', '미스터 주: 사라진 VIP', '오즈의 마법사: 요술구두와 말하는 책',
       '하이큐!! 땅 VS 하늘', '정직한 후보', '인셉션', '클로젯', '핑크퐁 시네마 콘서트 : 우주대탐험',
       '나이트 헌터', '지푸라기라도 잡고 싶은 짐승들', '버즈 오브 프레이(할리 퀸의 황홀한 해방)',
       '페인 앤 글로리', '조조 래빗', '작은 아씨들', '극장판 미니특공대: 공룡왕 디노',
       '이 멋진 세계에 축복을! 붉은 전설', '기생충', '수퍼 소닉', '극장판 원피스 스탬피드', '1917',
       '하이, 젝시', '숀더쉽 더 무비: 꼬마 외계인 룰라!', '신의 한 수: 귀수편', '젠틀맨', '인비저블맨',
       '해리포터와 아즈카반의 죄수', '시원찮은 그녀를 위한 육성방법 피날레', '장마', '엠마',
       '더 보이 2: 돌아온 브람스', '울프 콜', '어바웃 타임', '그링고', '다크 워터스', '스타 이즈 본',
       '메멘토', '걸즈 앤 판처 최종장', '쉰들러 리스트', '그린 북', '찬실이는 복도 많지', '세인트 아가타',
       '분노의 질주: 더 익스트림', '트루먼 쇼', '라라랜드', '스케어리 스토리:어둠의 속삭임', '주디',
       '사랑하고 있습니까', '페임', '날씨의 아이', '바이올렛 에버가든 – 영원과 

## 영화 코드를 넣으면 영화 장르랑 제목을 주는 함수

In [17]:
def movie_att(movie_code):
    address = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.xml?'
    option = 'key=5e9e89c5d971566b3955b179d41f94bd&movieCd='
    
    url = address + option + movie_code
    
    res = requests.get(url)
    html = res.content
    xmlobj = xmltodict.parse(html)
    
    movie_Nm_Gr = []  # 제목, 장르 합칠 곳
    
    movie_Name = xmlobj['movieInfoResult']['movieInfo']['movieNm']  # 영화 제목
    
    movie_Gr = []  # 영화 장르 들어갈곳
    for i in range(len(xmlobj['movieInfoResult']['movieInfo']['genres']['genre'])):
        if len(xmlobj['movieInfoResult']['movieInfo']['genres']['genre']) >= 2 :
            movie_Gr.append(xmlobj['movieInfoResult']['movieInfo']['genres']['genre'][i]['genreNm'])
        else:
            movie_Gr.append(xmlobj['movieInfoResult']['movieInfo']['genres']['genre']['genreNm'])
        
    movie_Nm_Gr.append(movie_Name)
    movie_Nm_Gr.append(movie_Gr)
    return movie_Nm_Gr

In [19]:
movie_att('20192206')

['백두산', ['어드벤처', '드라마']]

## 20년 상영작들을 대상으로 장르들을 구해보자

In [20]:
len(df['movieCd'].unique()) # 237개 영화가 상영됨

237

In [21]:
movie_Genre = []
for movie_code in df['movieCd'].unique():
    movie_Genre.append(movie_att(movie_code))
    print(movie_att(movie_code))

['백두산', ['어드벤처', '드라마']]
['천문: 하늘에 묻는다', ['사극']]
['시동', ['드라마']]
['미드웨이', ['액션', '드라마']]
['겨울왕국 2', ['애니메이션']]
['신비아파트 극장판 하늘도깨비 대 요르문간드', ['애니메이션']]
['포드 V 페라리', ['액션', '드라마']]
['21 브릿지: 테러 셧다운', ['스릴러', '드라마', '범죄']]
['나이브스 아웃', ['미스터리', '스릴러']]
['눈의 여왕4', ['애니메이션', '가족', '판타지', '어드벤처']]
['해치지않아', ['코미디']]
['프린스 코기', ['애니메이션']]
['닥터 두리틀', ['판타지', '어드벤처']]
['스타워즈: 라이즈 오브 스카이워커', ['액션', '어드벤처', '판타지', 'SF']]
['타발루가와 얼음공주', ['애니메이션', '어드벤처', '코미디']]
['스파이 지니어스', ['애니메이션', '액션', '코미디', '가족', 'SF']]
['나쁜 녀석들: 포에버', ['액션']]
['히트맨', ['코미디', '액션']]
['타오르는 여인의 초상', ['드라마']]
['아내를 죽였다', ['스릴러', '액션']]
['남산의 부장들', ['드라마']]
['미스터 주: 사라진 VIP', ['코미디', '드라마']]
['오즈의 마법사: 요술구두와 말하는 책', ['애니메이션', '코미디', '가족']]
['하이큐!! 땅 VS 하늘', ['애니메이션']]
['정직한 후보', ['코미디']]
['인셉션', ['드라마', '미스터리']]
['클로젯', ['미스터리', '드라마']]
['핑크퐁 시네마 콘서트 : 우주대탐험', ['애니메이션']]
['나이트 헌터', ['액션', '범죄', '스릴러', '드라마']]
['지푸라기라도 잡고 싶은 짐승들', ['스릴러', '범죄']]
['버즈 오브 프레이(할리 퀸의 황홀한 해방)', ['액션', '어드벤처', '범죄']]
['페인 앤 글로리', ['드라마']]
['조조 래빗', ['코

In [22]:
df['genre'] = ''

In [23]:
movie_Genre

[['백두산', ['어드벤처', '드라마']],
 ['천문: 하늘에 묻는다', ['사극']],
 ['시동', ['드라마']],
 ['미드웨이', ['액션', '드라마']],
 ['겨울왕국 2', ['애니메이션']],
 ['신비아파트 극장판 하늘도깨비 대 요르문간드', ['애니메이션']],
 ['포드 V 페라리', ['액션', '드라마']],
 ['21 브릿지: 테러 셧다운', ['스릴러', '드라마', '범죄']],
 ['나이브스 아웃', ['미스터리', '스릴러']],
 ['눈의 여왕4', ['애니메이션', '가족', '판타지', '어드벤처']],
 ['해치지않아', ['코미디']],
 ['프린스 코기', ['애니메이션']],
 ['닥터 두리틀', ['판타지', '어드벤처']],
 ['스타워즈: 라이즈 오브 스카이워커', ['액션', '어드벤처', '판타지', 'SF']],
 ['타발루가와 얼음공주', ['애니메이션', '어드벤처', '코미디']],
 ['스파이 지니어스', ['애니메이션', '액션', '코미디', '가족', 'SF']],
 ['나쁜 녀석들: 포에버', ['액션']],
 ['히트맨', ['코미디', '액션']],
 ['타오르는 여인의 초상', ['드라마']],
 ['아내를 죽였다', ['스릴러', '액션']],
 ['남산의 부장들', ['드라마']],
 ['미스터 주: 사라진 VIP', ['코미디', '드라마']],
 ['오즈의 마법사: 요술구두와 말하는 책', ['애니메이션', '코미디', '가족']],
 ['하이큐!! 땅 VS 하늘', ['애니메이션']],
 ['정직한 후보', ['코미디']],
 ['인셉션', ['드라마', '미스터리']],
 ['클로젯', ['미스터리', '드라마']],
 ['핑크퐁 시네마 콘서트 : 우주대탐험', ['애니메이션']],
 ['나이트 헌터', ['액션', '범죄', '스릴러', '드라마']],
 ['지푸라기라도 잡고 싶은 짐승들', ['스릴러', '범죄']],
 ['버즈 오브 프레이(할리 퀸의 황홀한 해방

In [24]:
for i in range(len(df['movieNm'])):
    for j in range(len(movie_Genre)):
        if df['movieNm'][i] == movie_Genre[j][0]:
            df['genre'][i] = movie_Genre[j][1]

In [25]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,"[어드벤처, 드라마]"
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439,[사극]
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,[드라마]
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,"[액션, 드라마]"
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,[애니메이션]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974,"[전쟁, 드라마]"
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628,"[코미디, 드라마]"
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314,[애니메이션]
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430,"[공포(호러), 스릴러]"


## 장르도 2700개 다 예쁘게 들어간 것을 확인

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2700 entries, 0 to 2699
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rank           2700 non-null   object
 1   rankInten      2700 non-null   object
 2   rankOldAndNew  2700 non-null   object
 3   movieCd        2700 non-null   object
 4   movieNm        2700 non-null   object
 5   openDt         2687 non-null   object
 6   salesAmt       2700 non-null   object
 7   salesShare     2700 non-null   object
 8   salesInten     2700 non-null   object
 9   salesChange    2700 non-null   object
 10  salesAcc       2700 non-null   object
 11  audiCnt        2700 non-null   object
 12  audiInten      2700 non-null   object
 13  audiChange     2700 non-null   object
 14  audiAcc        2700 non-null   object
 15  scrnCnt        2700 non-null   object
 16  showCnt        2700 non-null   object
 17  genre          2700 non-null   object
dtypes: object(18)
memory usage: 

In [27]:
abc = []
for data in df['genre']:
    abc.append(len(data))

## 최대 장르 개수가 5개까지 있는 것을 확인

In [28]:
abcd = set(abc)
abcd

{1, 2, 3, 4, 5}

In [29]:
df['genre2'] = ''
df['genre3'] = ''
df['genre4'] = ''
df['genre5'] = ''

In [30]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,"[어드벤처, 드라마]",,,,
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439,[사극],,,,
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,[드라마],,,,
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,"[액션, 드라마]",,,,
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,[애니메이션],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974,"[전쟁, 드라마]",,,,
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628,"[코미디, 드라마]",,,,
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314,[애니메이션],,,,
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430,"[공포(호러), 스릴러]",,,,


In [31]:
i = 0
for data in df['genre']:
    if len(data) == 5:
        df['genre5'][i] = data[4]
        df['genre4'][i] = data[3]
        df['genre3'][i] = data[2]
        df['genre2'][i] = data[1]
        df['genre'][i] = data[0]
    elif len(data) == 4:
        df['genre4'][i] = data[3]
        df['genre3'][i] = data[2]
        df['genre2'][i] = data[1]
        df['genre'][i] = data[0]
    elif len(data) == 3:
        df['genre3'][i] = data[2]
        df['genre2'][i] = data[1]
        df['genre'][i] = data[0]
    elif len(data) == 2:
        df['genre2'][i] = data[1]
        df['genre'][i] = data[0]
    elif len(data) == 1:
        df['genre'][i] = data[0]
    if i == 2699:
        break
    i += 1

In [32]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,어드벤처,드라마,,,
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439,사극,,,,
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,드라마,,,,
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,액션,드라마,,,
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,애니메이션,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974,전쟁,드라마,,,
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628,코미디,드라마,,,
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314,애니메이션,,,,
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430,공포(호러),스릴러,,,


## 날짜도 추가해주자

In [33]:
d = datetime.datetime(2020, 1, 1)

In [34]:
d.strftime('%Y-%m-%d')

'2020-01-01'

In [35]:
dt_index = pd.date_range(start = '20200101', end = '20200926')

In [36]:
date_list = []
for i in dt_index:
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)
    date_list.append(i)

In [37]:
len(date_list)

2700

In [38]:
df['date'] = date_list

In [39]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,어드벤처,드라마,,,,2020-01-01
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,239149,89345,59.6,1204773,929,3439,사극,,,,,2020-01-01
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,드라마,,,,,2020-01-01
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,액션,드라마,,,,2020-01-01
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,애니메이션,,,,,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,9944,4720,90.4,23443,454,974,전쟁,드라마,,,,2020-09-26
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,7751,3968,104.9,310803,384,628,코미디,드라마,,,,2020-09-26
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,3084,2330,309,4874,155,314,애니메이션,,,,,2020-09-26
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,2897,1406,94.3,84791,182,430,공포(호러),스릴러,,,,2020-09-26


In [40]:
df[df['movieNm']=='백두산']

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,어드벤처,드라마,,,,2020-01-01
10,1,0,OLD,20192206,백두산,2019-12-19,997483800,30.9,-3433485120,-77.5,58334242490,125013,-373789,-74.9,6914317,1241,5536,어드벤처,드라마,,,,2020-01-02
20,1,0,OLD,20192206,백두산,2019-12-19,1035750260,29.9,38266460,3.8,59369992750,118696,-6317,-5.1,7033013,1193,5565,어드벤처,드라마,,,,2020-01-03
30,1,0,OLD,20192206,백두산,2019-12-19,2128994980,30.5,1093244720,105.6,61498987730,241066,122370,103.1,7274079,1222,5709,어드벤처,드라마,,,,2020-01-04
40,1,0,OLD,20192206,백두산,2019-12-19,1809882740,30.5,-319112240,-15,63308870470,205310,-35756,-14.8,7479389,1215,5548,어드벤처,드라마,,,,2020-01-05
50,1,0,OLD,20192206,백두산,2019-12-19,576165280,29.1,-1233717460,-68.2,63885035750,71674,-133636,-65.1,7551063,1146,5088,어드벤처,드라마,,,,2020-01-06
60,1,0,OLD,20192206,백두산,2019-12-19,572810270,28.7,-3355010,-0.6,64457846020,71911,237,0.3,7622974,1171,5046,어드벤처,드라마,,,,2020-01-07
72,3,-2,OLD,20192206,백두산,2019-12-19,455463800,15.7,-117346470,-20.5,64913309820,56626,-15285,-21.3,7679600,864,3374,어드벤처,드라마,,,,2020-01-08
81,2,1,OLD,20192206,백두산,2019-12-19,411452030,17.0,-44011770,-9.7,65324761850,51441,-5185,-9.2,7731041,851,3230,어드벤처,드라마,,,,2020-01-09
91,2,0,OLD,20192206,백두산,2019-12-19,467789180,15.6,56337150,13.7,65792551030,52933,1492,2.9,7783974,881,3284,어드벤처,드라마,,,,2020-01-10


In [41]:
df['genre_union'] = ''
for i in range(len(df['movieNm'])):
    for j in range(len(movie_Genre)):
        if df['movieNm'][i] == movie_Genre[j][0]:
            df['genre_union'][i] = movie_Genre[j][1]

<ipython-input-41-26eaa67ab8d2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre_union'][i] = movie_Genre[j][1]


In [42]:
df

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,...,57.8,6789304,1236,5845,어드벤처,드라마,,,,2020-01-01,"[어드벤처, 드라마]"
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78,9900747970,...,59.6,1204773,929,3439,사극,,,,,2020-01-01,[사극]
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,...,39.8,2716441,801,2997,드라마,,,,,2020-01-01,[드라마]
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,...,32.2,307780,810,2614,액션,드라마,,,,2020-01-01,"[액션, 드라마]"
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,...,50.6,13452593,665,1459,애니메이션,,,,,2020-01-01,[애니메이션]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,...,90.4,23443,454,974,전쟁,드라마,,,,2020-09-26,"[전쟁, 드라마]"
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,2498196430,...,104.9,310803,384,628,코미디,드라마,,,,2020-09-26,"[코미디, 드라마]"
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,35855370,...,309,4874,155,314,애니메이션,,,,,2020-09-26,[애니메이션]
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,754542800,...,94.3,84791,182,430,공포(호러),스릴러,,,,2020-09-26,"[공포(호러), 스릴러]"


In [43]:
df_movie_one = [] ## 가장 마지막에 상영된 날짜의 정보
for movie_name in df['movieNm'].unique():
    movie = df[(df['movieNm'] == movie_name) & (df['date'] == max(df[df['movieNm'] == movie_name]['date']))]
    df_movie_one.append(movie)

In [44]:
len(df_movie_one) # 20년 개봉한 영화 종류 수

237

In [45]:
df_movie_one_result = pd.DataFrame()  # 마지막 상영된 날짜의 영화 리스트
for i in range(len(df_movie_one)):
    df_movie_one_result = df_movie_one_result.append(df_movie_one[i])

In [46]:
df_movie_one_result

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
259,10,-1,OLD,20192206,백두산,2019-12-19,32730680,0.3,-6141440,-15.8,69747400040,...,-12.2,8242478,120,146,어드벤처,드라마,,,,2020-01-26,"[어드벤처, 드라마]"
249,10,1,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,19061960,0.2,4647260,32.2,16442011940,...,23.2,1987842,89,104,사극,,,,,2020-01-25,[사극]
209,10,0,OLD,20198414,시동,2019-12-18,24186580,1.6,779540,3.3,28079682000,...,0.7,3314658,295,489,드라마,,,,,2020-01-21,[드라마]
1378,9,0,NEW,20196272,미드웨이,2019-12-31,10000000,1.7,10000000,100,8227133530,...,100,957894,10,10,액션,드라마,,,,2020-05-17,"[액션, 드라마]"
269,10,2,OLD,20197803,겨울왕국 2,2019-11-21,25100660,0.2,1806340,7.8,114733135850,...,10.7,13737960,56,75,애니메이션,,,,,2020-01-27,[애니메이션]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,3,-1,OLD,20190601,디바,2020-09-23,207832680,14.2,79966000,62.5,569726280,...,54.7,68347,811,3128,스릴러,미스터리,,,,2020-09-26,"[스릴러, 미스터리]"
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,196366260,...,90.4,23443,454,974,전쟁,드라마,,,,2020-09-26,"[전쟁, 드라마]"
2669,10,0,NEW,20181162,돌멩이,None,2205000,0.4,2205000,100,2205000,...,100,245,5,5,드라마,,,,,2020-09-23,[드라마]
2693,4,0,OLD,20204684,브레이크 더 사일런스: 더 무비,2020-09-24,203256500,13.9,75746500,59.4,517789500,...,66.6,57363,199,1059,다큐멘터리,,,,,2020-09-26,[다큐멘터리]


In [47]:
df_movie_one_result['salesAcc'] = pd.to_numeric(df_movie_one_result['salesAcc'], downcast = 'float')

In [48]:
df_movie_one_result

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
259,10,-1,OLD,20192206,백두산,2019-12-19,32730680,0.3,-6141440,-15.8,6.974740e+10,...,-12.2,8242478,120,146,어드벤처,드라마,,,,2020-01-26,"[어드벤처, 드라마]"
249,10,1,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,19061960,0.2,4647260,32.2,1.644201e+10,...,23.2,1987842,89,104,사극,,,,,2020-01-25,[사극]
209,10,0,OLD,20198414,시동,2019-12-18,24186580,1.6,779540,3.3,2.807968e+10,...,0.7,3314658,295,489,드라마,,,,,2020-01-21,[드라마]
1378,9,0,NEW,20196272,미드웨이,2019-12-31,10000000,1.7,10000000,100,8.227133e+09,...,100,957894,10,10,액션,드라마,,,,2020-05-17,"[액션, 드라마]"
269,10,2,OLD,20197803,겨울왕국 2,2019-11-21,25100660,0.2,1806340,7.8,1.147331e+11,...,10.7,13737960,56,75,애니메이션,,,,,2020-01-27,[애니메이션]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,3,-1,OLD,20190601,디바,2020-09-23,207832680,14.2,79966000,62.5,5.697263e+08,...,54.7,68347,811,3128,스릴러,미스터리,,,,2020-09-26,"[스릴러, 미스터리]"
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,1.963663e+08,...,90.4,23443,454,974,전쟁,드라마,,,,2020-09-26,"[전쟁, 드라마]"
2669,10,0,NEW,20181162,돌멩이,None,2205000,0.4,2205000,100,2.205000e+06,...,100,245,5,5,드라마,,,,,2020-09-23,[드라마]
2693,4,0,OLD,20204684,브레이크 더 사일런스: 더 무비,2020-09-24,203256500,13.9,75746500,59.4,5.177895e+08,...,66.6,57363,199,1059,다큐멘터리,,,,,2020-09-26,[다큐멘터리]


In [49]:
pd.options.display.float_format = '{:.1f}'.format

In [50]:
df_movie_one_result['salesAcc'].sort_values(ascending = False)[:20]  # 탑 20개 누적매출액
top20_sales = df_movie_one_result['salesAcc'].sort_values(ascending = False)[:20].index

In [51]:
top20_sales

Int64Index([1257,  269, 2309, 1249, 1184, 1208,  259, 1185, 1953, 2379, 1099,
            2235,  549, 2678, 1675, 2424, 1327,  819,  918, 1883],
           dtype='int64')

In [52]:
df.loc[top20_sales] # 그런데 보니까 대부분의 경우가 2020년 이전에 개봉한것. 즉 개봉일이 2020년 이후인 것중에서만 뽑아보자

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
1257,8,-2,OLD,20184889,어벤져스: 엔드게임,2019-04-24,47233940,5.1,21758700,85.4,122492181020,...,90.1,13977602,439,587,액션,SF,,,,2020-05-05,"[액션, SF]"
269,10,2,OLD,20197803,겨울왕국 2,2019-11-21,25100660,0.2,1806340,7.8,114733135850,...,10.7,13737960,56,75,애니메이션,,,,,2020-01-27,[애니메이션]
2309,10,1,OLD,20183867,알라딘,2019-05-23,4301000,0.4,-4821000,-52.9,108160426039,...,-52.7,12723757,28,55,어드벤처,가족,판타지,,,2020-08-18,"[어드벤처, 가족, 판타지]"
1249,10,0,OLD,20177478,어벤져스: 인피니티 워,2018-04-25,11147060,3.0,-5616180,-33.5,100063910649,...,-33.5,11230721,286,351,액션,어드벤처,판타지,SF,,2020-05-04,"[액션, 어드벤처, 판타지, SF]"
1184,5,1,OLD,20148493,어벤져스: 에이지 오브 울트론,2015-04-23,6327920,3.9,1128240,21.7,88638992746,...,26.1,10503764,291,387,액션,어드벤처,판타지,SF,,2020-04-28,"[액션, 어드벤처, 판타지, SF]"
1208,9,-3,OLD,20183782,기생충,2019-05-30,38902340,4.6,-1725780,-4.2,87354331615,...,-26.5,10300613,447,922,드라마,,,,,2020-04-30,[드라마]
259,10,-1,OLD,20192206,백두산,2019-12-19,32730680,0.3,-6141440,-15.8,69747400040,...,-12.2,8242478,120,146,어드벤처,드라마,,,,2020-01-26,"[어드벤처, 드라마]"
1185,6,-1,OLD,20124721,어벤져스,2012-04-26,5897580,3.6,210660,3.7,59633952458,...,7.4,7086884,277,376,액션,SF,,,,2020-04-28,"[액션, SF]"
1953,4,1,OLD,20112703,다크 나이트 라이즈,2012-07-19,18218920,4.9,2360440,14.9,47908099120,...,14.2,6428134,280,443,액션,범죄,스릴러,,,2020-07-14,"[액션, 범죄, 스릴러]"
2379,10,1,OLD,20100312,인셉션,2010-07-21,7132980,1.5,2226980,45.4,44857075280,...,41.8,5997867,36,78,드라마,미스터리,,,,2020-08-25,"[드라마, 미스터리]"


In [53]:
df[df['openDt'].isnull()]

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
1916,7,5,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,12000000,2.0,5400000,81.8,18600000,...,81.8,3100,4,19,기타,,,,,2020-07-10,[기타]
1927,8,-1,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,18600000,1.5,6600000,55,37200000,...,55,6200,6,28,기타,,,,,2020-07-11,[기타]
1938,9,-1,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,15600000,1.4,-3000000,-16.1,52800000,...,-16.1,8800,5,23,기타,,,,,2020-07-12,[기타]
1946,7,2,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,8400000,2.1,-7200000,-46.2,61200000,...,-46.2,10200,3,12,기타,,,,,2020-07-13,[기타]
1957,8,-1,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,8400000,2.2,0,0,69600000,...,0,11600,3,12,기타,,,,,2020-07-14,[기타]
1962,3,5,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,26400000,0.9,18000000,214.3,96000000,...,214.3,16000,8,38,기타,,,,,2020-07-15,[기타]
1972,3,0,OLD,20201421,BIFAN2020 판타스틱 단편 걸작선 1,None,24000000,1.2,-2400000,-9.1,120000000,...,-9.1,20000,8,35,기타,,,,,2020-07-16,[기타]
2116,7,0,NEW,20200144,기묘한 이야기: 리플레이,None,1062000,0.1,1062000,100,1063000,...,100,1045,1,3,어드벤처,공포(호러),,,,2020-07-30,"[어드벤처, 공포(호러)]"
2117,8,0,NEW,20209466,죽이는 여자,None,876000,0.1,876000,100,1481000,...,100,1463,1,2,공포(호러),,,,,2020-07-30,[공포(호러)]
2626,7,0,NEW,20205386,인디애니페스트2020 릴레이작가 초청전,None,8220000,0.6,8220000,100,8766000,...,100,1461,4,10,애니메이션,,,,,2020-09-19,[애니메이션]


In [54]:
open_Dt = df['openDt'].copy()
null_index = open_Dt[open_Dt.isnull()].index
null_index

Int64Index([1916, 1927, 1938, 1946, 1957, 1962, 1972, 2116, 2117, 2626, 2637,
            2658, 2669],
           dtype='int64')

In [55]:
open_Dt[1916] = '2020-07-10'  # 날짜 기입 안되어있는 항목들에 날짜 넣어줌
open_Dt[1927] = '2020-07-10'
open_Dt[1938] = '2020-07-10'
open_Dt[1946] = '2020-07-10'
open_Dt[1916] = '2020-07-10'
open_Dt[1957] = '2020-07-10'
open_Dt[1962] = '2020-07-10'
open_Dt[1972] = '2020-07-10'
open_Dt[2116] = '2020-06-25'
open_Dt[2117] = '2016-02-12'
open_Dt[2626] = '2020-09-19'
open_Dt[2637] = '2020-09-19'
open_Dt[2658] = '2020-09-19'
open_Dt[2669] = '2018-10-05'

In [56]:
df['openDt'] = open_Dt

In [57]:
date_set = []  # str에서 datetime으로 바꿔준다
for date in df['openDt']:
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    date_set.append(date)

In [58]:
df['openDt'] = date_set

In [59]:
df.info() # 정상적으로 datetime으로 바뀜

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2700 entries, 0 to 2699
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rank           2700 non-null   object        
 1   rankInten      2700 non-null   object        
 2   rankOldAndNew  2700 non-null   object        
 3   movieCd        2700 non-null   object        
 4   movieNm        2700 non-null   object        
 5   openDt         2700 non-null   datetime64[ns]
 6   salesAmt       2700 non-null   object        
 7   salesShare     2700 non-null   object        
 8   salesInten     2700 non-null   object        
 9   salesChange    2700 non-null   object        
 10  salesAcc       2700 non-null   object        
 11  audiCnt        2700 non-null   object        
 12  audiInten      2700 non-null   object        
 13  audiChange     2700 non-null   object        
 14  audiAcc        2700 non-null   object        
 15  scrnCnt        2700 n

In [60]:
df['salesAcc'] = pd.to_numeric(df['salesAcc'], downcast = 'float')

In [61]:
df_movie_one = [] ## 가장 마지막에 상영된 날짜의 정보
for movie_name in df['movieNm'].unique():
    movie = df[(df['movieNm'] == movie_name) & (df['date'] == max(df[df['movieNm'] == movie_name]['date']))]
    df_movie_one.append(movie)

df_movie_one_result = pd.DataFrame()  # 마지막 상영된 날짜의 영화 리스트
for i in range(len(df_movie_one)):
    df_movie_one_result = df_movie_one_result.append(df_movie_one[i])

In [65]:
df_open20 = df_movie_one_result[df_movie_one_result['openDt'] >= datetime.datetime(2020, 1, 1)]

In [71]:
df_open20_top20_index = df_open20['salesAcc'].sort_values(ascending = False)[:20].index  # 20년 개봉작들중 누적 매출액 Top 20의 index

In [74]:
df_open20_top20 = df.loc[df_open20_top20_index]  # 20년 개봉작들중 누적 매출액 Top 20
df_open20_top20

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,...,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
549,10,1,OLD,20180939,남산의 부장들,2020-01-22,3080960,0.5,-5089720,-62.3,41209708544.0,...,-57.9,4748387,94,135,드라마,,,,,2020-02-24,[드라마]
2678,9,5,OLD,20197922,다만 악에서 구하소서,2020-08-05,8907980,1.3,7445880,509.3,38572023808.0,...,850.3,4353356,39,50,범죄,액션,,,,2020-09-24,"[범죄, 액션]"
2424,5,0,OLD,20193450,반도,2020-07-15,9627000,0.6,2253500,30.6,33059989504.0,...,32,3810380,50,55,액션,드라마,,,,2020-08-30,"[액션, 드라마]"
488,9,0,OLD,20192101,히트맨,2020-01-22,15765000,0.9,189780,1.2,20601798656.0,...,4.3,2404799,285,406,코미디,액션,,,,2020-02-18,"[코미디, 액션]"
2425,6,0,OLD,20193069,#살아있다,2020-06-24,4445000,0.3,-1266000,-22.2,15965329408.0,...,-22.2,1903703,5,5,드라마,,,,,2020-08-30,[드라마]
2690,1,0,OLD,20201122,테넷,2020-08-26,362360370,24.7,207354590,133.8,15162626048.0,...,133.9,1646895,859,2367,액션,SF,,,,2020-09-26,"[액션, SF]"
2699,10,0,OLD,20196271,강철비2: 정상회담,2020-07-29,5834000,0.4,3033000,108.3,14649988096.0,...,108,1788397,9,16,드라마,액션,,,,2020-09-26,"[드라마, 액션]"
329,10,1,OLD,20199581,닥터 두리틀,2020-01-08,16408200,0.5,-2356570,-12.6,13430496256.0,...,-12.8,1604127,204,281,판타지,어드벤처,,,,2020-02-02,"[판타지, 어드벤처]"
907,8,2,OLD,20193321,정직한 후보,2020-02-12,6510900,2.4,2446660,60.2,12804876288.0,...,60.9,1531010,161,222,코미디,,,,,2020-03-31,[코미디]
2638,9,-3,OLD,20199883,오케이 마담,2020-08-12,5942500,0.5,-2986000,-33.4,10978805760.0,...,-31.3,1228551,23,26,코미디,액션,,,,2020-09-20,"[코미디, 액션]"


In [85]:
df_open20_top20 = df_open20_top20.sort_values(by = ['salesAcc'], axis = 0, ascending= False)
df_open20_top20

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
549,10,1,OLD,20180939,남산의 부장들,2020-01-22,3080960,0.5,-5089720,-62.3,41209708544.0,382,-526,-57.9,4748387,94,135,드라마,,,,,2020-02-24,[드라마]
2678,9,5,OLD,20197922,다만 악에서 구하소서,2020-08-05,8907980,1.3,7445880,509.3,38572023808.0,1701,1522,850.3,4353356,39,50,범죄,액션,,,,2020-09-24,"[범죄, 액션]"
2424,5,0,OLD,20193450,반도,2020-07-15,9627000,0.6,2253500,30.6,33059989504.0,1849,448,32,3810380,50,55,액션,드라마,,,,2020-08-30,"[액션, 드라마]"
488,9,0,OLD,20192101,히트맨,2020-01-22,15765000,0.9,189780,1.2,20601798656.0,2039,84,4.3,2404799,285,406,코미디,액션,,,,2020-02-18,"[코미디, 액션]"
2425,6,0,OLD,20193069,#살아있다,2020-06-24,4445000,0.3,-1266000,-22.2,15965329408.0,889,-253,-22.2,1903703,5,5,드라마,,,,,2020-08-30,[드라마]
2690,1,0,OLD,20201122,테넷,2020-08-26,362360370,24.7,207354590,133.8,15162626048.0,38603,22101,133.9,1646895,859,2367,액션,SF,,,,2020-09-26,"[액션, SF]"
2699,10,0,OLD,20196271,강철비2: 정상회담,2020-07-29,5834000,0.4,3033000,108.3,14649988096.0,2877,1494,108,1788397,9,16,드라마,액션,,,,2020-09-26,"[드라마, 액션]"
329,10,1,OLD,20199581,닥터 두리틀,2020-01-08,16408200,0.5,-2356570,-12.6,13430496256.0,2456,-359,-12.8,1604127,204,281,판타지,어드벤처,,,,2020-02-02,"[판타지, 어드벤처]"
907,8,2,OLD,20193321,정직한 후보,2020-02-12,6510900,2.4,2446660,60.2,12804876288.0,1036,392,60.9,1531010,161,222,코미디,,,,,2020-03-31,[코미디]
2638,9,-3,OLD,20199883,오케이 마담,2020-08-12,5942500,0.5,-2986000,-33.4,10978805760.0,1135,-516,-31.3,1228551,23,26,코미디,액션,,,,2020-09-20,"[코미디, 액션]"


## 코로나 이전 년도와 비교하기위해 18년 19년 같은 월의 정보들을 가져오자

In [86]:
movie_sep18 = movie_info(20180901, 20180926)  # 18년
movie_aug18 = movie_info(20180801, 20180831)
movie_Jul18 = movie_info(20180701, 20180731)
movie_Jun18 = movie_info(20180601, 20180630)
movie_May18 = movie_info(20180501, 20180531)
movie_Apr18 = movie_info(20180401, 20180430)
movie_Mar18 = movie_info(20180301, 20180331)
movie_Feb18 = movie_info(20180201, 20180228)
movie_Jan18 = movie_info(20180101, 20180131)

In [115]:
movie_sep19 = movie_info(20190901, 20190926)  # 19년
movie_aug19 = movie_info(20190801, 20190831)
movie_Jul19 = movie_info(20190701, 20190731)
movie_Jun19 = movie_info(20190601, 20190630)
movie_May19 = movie_info(20190501, 20190531)
movie_Apr19 = movie_info(20190401, 20190430)
movie_Mar19 = movie_info(20190301, 20190331)
movie_Feb19 = movie_info(20190201, 20190228)
movie_Jan19 = movie_info(20190101, 20190131)

In [116]:
movie_19 = [movie_Jan19, movie_Feb19, movie_Mar19, movie_Apr19, movie_May19, movie_Jun19, movie_Jul19, movie_aug19, movie_sep19]
movie_18 = [movie_Jan18, movie_Feb18, movie_Mar18, movie_Apr18, movie_May18, movie_Jun18, movie_Jul18, movie_aug18, movie_sep18]

In [102]:
df_18 = pd.DataFrame(columns = col_name)
j = 0
for i in range(len(movie_18)):  #월 수만큼 반복
    for source in movie_18[i]:  #각 월마다 일수들을 source로
        source = list(source)   #리스트화해서 넣기
        df_18.loc[j] = source
        j += 1

In [117]:
df_19 = pd.DataFrame(columns = col_name)
j = 0
for i in range(len(movie_19)):  #월 수만큼 반복
    for source in movie_19[i]:  #각 월마다 일수들을 source로
        source = list(source)   #리스트화해서 넣기
        df_19.loc[j] = source
        j += 1

In [118]:
df_18.head(5)

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20150976,신과함께-죄와 벌,2017-12-20,7715149100,52.9,-853021100,-10,76065645532,916652,-107305,-10.5,9456147,1644,7957
1,2,2,0,OLD,20170590,1987,2017-12-27,4460462100,30.6,-224621200,-4.8,19714466137,531469,-28577,-5.1,2473546,1097,4982
2,3,3,0,OLD,20170402,강철비,2017-12-14,944228600,6.5,-48816300,-4.9,32939126166,114298,-6548,-5.4,4128591,602,1451
3,4,4,0,OLD,20179462,위대한 쇼맨,2017-12-20,509947700,3.5,25348100,5.2,7166974900,60070,2402,4.2,874655,432,843
4,5,5,0,OLD,20174965,극장판 포켓몬스터 너로 정했다!,2017-12-21,262886500,1.8,-7375400,-2.7,3380416200,34103,-1037,-3,449358,417,605


In [119]:
df_19.head(5)

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20180290,아쿠아맨,2018-12-19,3482411050,33.8,1154385750,49.6,33936703431,384106,96412,33.5,3875963,1158,4594
1,2,2,0,OLD,20170658,PMC: 더 벙커,2018-12-26,1763943100,17.1,425397650,31.8,11468190467,195859,30733,18.6,1374890,893,3530
2,3,3,1,OLD,20185485,보헤미안 랩소디,2018-10-31,1236738450,12.0,378199650,44.1,81247178795,135879,29431,27.6,9360461,732,1872
3,4,4,-1,OLD,20186281,범블비,2018-12-25,1143504700,11.1,264548150,30.1,10899588452,132618,21611,19.5,1299768,766,2519
4,5,5,0,NEW,20184187,언니,2019-01-01,612798450,5.9,612798450,100,623953450,66686,66686,100,67929,509,1679


In [120]:
movie_Genre = []
for movie_code in df_18['movieCd'].unique():
    movie_Genre.append(movie_att(movie_code))

df_18['genre'] = ''

for i in range(len(df_18['movieNm'])):
    for j in range(len(movie_Genre)):
        if df_18['movieNm'][i] == movie_Genre[j][0]:
            df_18['genre'][i] = movie_Genre[j][1]

In [25]:
movie_Genre = []
for movie_code in df_19['movieCd'].unique():
    movie_code = str(movie_code)   # csv로 저장후 가져오니까 형식이 int로 바뀌어서 str로 다시 바꿔줌
    movie_Genre.append(movie_att(movie_code))

df_19['genre'] = ''

for i in range(len(df_19['movieNm'])):
    for j in range(len(movie_Genre)):
        if df_19['movieNm'][i] == movie_Genre[j][0]:
            df_19['genre'][i] = movie_Genre[j][1]

<ipython-input-25-dd888ace6732>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_19['genre'][i] = movie_Genre[j][1]


In [3]:
df_20 = pd.read_csv('C:/Users/msi/Desktop/빅데이터/20_all.csv', parse_dates=['openDt', 'date'])
df_19 = pd.read_csv('C:/Users/msi/Desktop/빅데이터/19_all.csv', parse_dates=['openDt'])
df_18 = pd.read_csv('C:/Users/msi/Desktop/빅데이터/18_all.csv', parse_dates=['openDt'])

In [52]:
dt_index = pd.date_range(start = '20180101', end = '20180926')
df_18['date'] = ''

dt_list = []
for dt in dt_index:
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    
df_18['date'] = dt_list

In [5]:
dt_index = pd.date_range(start = '20190101', end = '20190926')
df_19['date'] = ''

dt_list = []
for dt in dt_index:
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    dt_list.append(dt)
    
df_19['date'] = dt_list

In [8]:
pd.options.display.max_columns = 20

In [31]:
df_20

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,...,audiAcc,scrnCnt,showCnt,genre,genre2,genre3,genre4,genre5,date,genre_union
0,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,...,6789304,1236,5845,어드벤처,드라마,NaN,NaN,NaN,2020-01-01,"['어드벤처', '드라마']"
1,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78.0,...,1204773,929,3439,사극,NaN,NaN,NaN,NaN,2020-01-01,['사극']
2,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,...,2716441,801,2997,드라마,NaN,NaN,NaN,NaN,2020-01-01,['드라마']
3,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,...,307780,810,2614,액션,드라마,NaN,NaN,NaN,2020-01-01,"['액션', '드라마']"
4,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,...,13452593,665,1459,애니메이션,NaN,NaN,NaN,NaN,2020-01-01,['애니메이션']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,6,0,OLD,20202341,아웃포스트,2020-09-23,87542020,6.0,43137150,97.1,...,23443,454,974,전쟁,드라마,NaN,NaN,NaN,2020-09-26,"['전쟁', '드라마']"
2696,7,0,OLD,20193666,오! 문희,2020-09-02,63239260,4.3,38231700,152.9,...,310803,384,628,코미디,드라마,NaN,NaN,NaN,2020-09-26,"['코미디', '드라마']"
2697,8,4,OLD,20206178,극장판 엉덩이 탐정: 텐텐마을의 수수께끼,2020-09-24,22909370,1.6,17205150,301.6,...,4874,155,314,애니메이션,NaN,NaN,NaN,NaN,2020-09-26,['애니메이션']
2698,9,0,OLD,20201687,기기괴괴 성형수,2020-09-09,26771100,1.8,13150000,96.5,...,84791,182,430,공포(호러),스릴러,NaN,NaN,NaN,2020-09-26,"['공포(호러)', '스릴러']"


In [59]:
df_19

,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,...,audiAcc,scrnCnt,showCnt,date,genre,genre2,genre3,genre4,genre5,genre_union
0,1,0,OLD,20180290,아쿠아맨,2018-12-19,3482411050,33.8,1154385750,49.6,...,3875963,1158,4594,2019-01-01,액션,,,,,[액션]
1,2,0,OLD,20170658,PMC: 더 벙커,2018-12-26,1763943100,17.1,425397650,31.8,...,1374890,893,3530,2019-01-01,액션,,,,,[액션]
2,3,1,OLD,20185485,보헤미안 랩소디,2018-10-31,1236738450,12.0,378199650,44.1,...,9360461,732,1872,2019-01-01,드라마,,,,,[드라마]
3,4,-1,OLD,20186281,범블비,2018-12-25,1143504700,11.1,264548150,30.1,...,1299768,766,2519,2019-01-01,액션,어드벤처,SF,,,"[액션, 어드벤처, SF]"
4,5,0,NEW,20184187,언니,2019-01-01,612798450,5.9,612798450,100.0,...,67929,509,1679,2019-01-01,액션,,,,,[액션]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,6,0,OLD,20184572,타짜: 원 아이드 잭,2019-09-11,62133940,3.3,-26242080,-29.7,...,2182407,424,1018,2019-09-26,범죄,드라마,,,,"[범죄, 드라마]"
2686,7,0,OLD,20195968,더 룸,2019-09-25,41016200,2.2,-19544700,-32.3,...,19993,144,572,2019-09-26,미스터리,스릴러,,,,"[미스터리, 스릴러]"
2687,8,1,OLD,20195203,레플리카,2019-09-25,38023670,2.0,-14678320,-27.9,...,13565,344,838,2019-09-26,범죄,미스터리,SF,스릴러,,"[범죄, 미스터리, SF, 스릴러]"
2688,9,-1,OLD,20193807,예스터데이,2019-09-18,33855500,1.8,-21213500,-38.5,...,271701,338,541,2019-09-26,드라마,,,,,[드라마]


In [47]:
df_18['genre2'] = ''
df_18['genre3'] = ''
df_18['genre4'] = ''
df_18['genre5'] = ''
df_18['genre_union'] = df_18['genre']
i = 0
for data in df_18['genre']:
    if len(data) == 5:
        df_18['genre5'][i] = data[4]
        df_18['genre4'][i] = data[3]
        df_18['genre3'][i] = data[2]
        df_18['genre2'][i] = data[1]
        df_18['genre'][i] = data[0]
    elif len(data) == 4:
        df_18['genre4'][i] = data[3]
        df_18['genre3'][i] = data[2]
        df_18['genre2'][i] = data[1]
        df_18['genre'][i] = data[0]
    elif len(data) == 3:
        df_18['genre3'][i] = data[2]
        df_18['genre2'][i] = data[1]
        df_18['genre'][i] = data[0]
    elif len(data) == 2:
        df_18['genre2'][i] = data[1]
        df_18['genre'][i] = data[0]
    elif len(data) == 1:
        df_18['genre'][i] = data[0]
    if i == 2699:
        break
    i += 1